<a href="https://colab.research.google.com/github/mykhalevychv/emotion_recognition/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten,AveragePooling2D
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score
from keras.optimizers import Adam, SGD
from keras import Sequential
from keras.utils import plot_model
import cv2
import os
import warnings
import tensorflow as tf

In [ ]:
%load_ext autoreload
%autoreload 2
  
from google.colab import drive
import sys
from pathlib import Path
drive.mount("/content/drive", force_remount=True)
base = Path('/content/drive/My Drive/project/')
sys.path.append(str(base))


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/project/
train_dir = '/content/drive/MyDrive/project/fer2013/train/'
test_dir = '/content/drive/MyDrive/project/fer2013/test/'

In [ ]:
print('training pictures\n')
import matplotlib.pyplot as plt
plt.figure(figsize=(14,22))
i = 1
for expression in os.listdir(train_dir):
    img = load_img((train_dir + expression +'/'+ os.listdir(train_dir + expression)[3]))
    plt.subplot(1,7,i)
    plt.imshow(img)
    plt.title(expression)
    plt.axis('off')
    i += 1
plt.show()

In [ ]:
row, col = 48, 48
classes = 7

def count_exp(path, set_):
    dict_ = {}
    for expression in os.listdir(path):
        dir_ = path + expression
        dict_[expression] = len(os.listdir(dir_))
    df = pd.DataFrame(dict_, index=[set_])
    return df
train_count = count_exp(train_dir, 'train')
test_count = count_exp(test_dir, 'test')
print(train_count)
print(test_count)

       angry  disgust  fear  happy  neutral   sad  surprise
train   3995      436  5108   7215     4965  4830      3171
      angry  disgust  fear  happy  neutral   sad  surprise
test    958      111  1024   1774     1233  1247       831


In [ ]:
train_datagen = ImageDataGenerator(#rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         #brightness_range = [0.1,1.1],
                                         horizontal_flip = True,
                                         #vertical_flip = True,
                                         rescale = 1./255,
                                         zoom_range = 0.2,
                                         validation_split = 0.2
                                        )
validation_datagen =ImageDataGenerator(rescale = 1./255,
                                             validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                               target_size = (48,48),
                                                               color_mode = "grayscale",
                                                               class_mode = "categorical",
                                                               batch_size = 64,
                                                               #shuffle = False,
                                                               subset = "training"
)

validation_generator = validation_datagen.flow_from_directory(directory = test_dir,
                                                             target_size = (48,48),
                                                             color_mode = "grayscale",
                                                             class_mode = "categorical",
                                                             batch_size = 64,
                                                             #shuffle = True,
                                                             subset = "validation"
)

Found 29720 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(48, 48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

In [ ]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('./weights.h5', monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from tensorflow.keras import layers
loss='categorical_crossentropy'
from tensorflow.keras import regularizers
optimizer = 'rmsprop'          
metrics = ['accuracy']

def cnn_model(conv_activation='relu', dropout_rate=0.25, optimizer='rmsprop',learning_rate=0.001,n = 512, epochs=10):
    

    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape = (48,48,1),activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3),activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))      

    model.add(Conv2D(128, (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3),activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))    
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))  

    model.add(Conv2D(256, (3, 3),activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3),activation = 'relu'))
    model.add(BatchNormalization())       
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate)) 

    model.add(Flatten())
    model.add(Dense(n))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=0.25))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    if optimizer == 'rmsprop':
        opt = RMSprop(lr=learning_rate)
    if optimizer == 'adam':
        opt = Adam(lr=learning_rate)
    if optimizer == 'sgd':
        opt = SGD(lr=learning_rate)
    if optimizer == 'adamax':
        opt = Adamax(lr=learning_rate)
    model.compile(optimizer=opt,  loss=loss,  metrics=['accuracy'])
    return model

epochs = 10
batch_size = 64
 
cnn_1 = cnn_model()

#cnn_model.compile(optimizer=opt,  loss=loss,  metrics=['accuracy'])

cnn_1.summary()

In [ ]:
plot_model(cnn_1, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
hist_model_cnn = cnn_1.fit(train_generator, steps_per_epoch =22968//64, epochs = 60,validation_data = validation_generator,validation_steps = 1432//64,  callbacks=[lr_reducer, checkpointer, early_stopper])

In [ ]:
cnn_1.save('./model.h5')

In [ ]:
y_pred = k.predict(train_generator)
y_pred = np.argmax(y_pred, axis=1)
class_labels = validation_generator.class_indices
class_labels = {v:k for k,v in class_labels.items()}

from sklearn.metrics import classification_report, confusion_matrix
cm_train = confusion_matrix(train_generator.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(train_generator.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

In [ ]:
y_pred = k.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = test_set.class_indices
class_labels = {v:k for k,v in class_labels.items()}

from sklearn.metrics import classification_report, confusion_matrix
cm_train = confusion_matrix(test_set.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(test_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

In [ ]:
print("rmsprop, lr = 0,001, 60 epochs")
#  "Accuracy"
plt.plot(hist_model_cnn.history['accuracy'])
plt.plot(hist_model_cnn.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(hist_model_cnn.history['loss'])
plt.plot(hist_model_cnn.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
train_loss, train_acc = k.evaluate(train_generator)
test_loss, test_acc   = k.evaluate(test_set)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_acc*100, test_acc*100))

In [ ]:
(X_train, Y_train) = train_generator.next()

In [ ]:
from datetime import datetime
from time import time, sleep
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
# optimize model 
start = time()
n_epochs=10
# define function to display the results of the grid search
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))  
# create model
model = KerasClassifier(build_fn=cnn_model, verbose=1)
# define parameters and values for grid search 
param_grid = {
    'dropout_rate' : [0.25, 0.5],
    'conv_activation': ['relu','tahn','sigmoid'],    
    'n':[256,512],
    #'epochs': [n_epochs],
    'optimizer': ['rmsprop','adam','sgd',],
    'learning_rate': [0.01, 0.001, 0.0001]
}
n_cv = 5
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=n_cv)
grid_result = grid.fit(X_train, Y_train)

# summarize results
print('time for grid search = {:.0f} sec'.format(time()-start))
display_cv_results(grid_result)

In [ ]:
#visualize filters from first layer
from matplotlib import pyplot
# retrieve weights from the second hidden layer
filters, biases = cnn_1.layers[1].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
#pyplot.figure(figsize=(1,1))
pyplot.figure(figsize=(10, 8), dpi=100)
n_filters, ix = 6, 1
for i in range(n_filters):
	# get the filter
	f = filters[:, :, :, i]
	# plot each channel separately figure(figsize=(8, 6), dpi=80)
	for j in range(3):
		# specify subplot and turn of axis
		ax = pyplot.subplot(n_filters, 3, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		pyplot.imshow(f[:, :, j], cmap='gray')
		ix += 1
# show the figure
pyplot.show()